In [1]:
import pandas as pd
import numpy as np
import matplotlib

from readCanLitefileUtils import *
from calibUtils import *

In [2]:
# define the inputs

#file 1 (DACset 0x000)
file_DACset000 = './files/21-05-06_160653_calib_SN04_DACSet000_20x.csv'

#file 2 (DACset 0xFFF)
file_DACsetFFF = './files/21-05-06_160653_calib_SN04_DACSetFFF_20x.csv'

DMMmin = [108.752, 108.755, 108.74, 108.744, 108.744, 108.744, 
          108.735, 108.741, 108.745, 108.715, 108.726, 108.755]

DMMmax = [116.327, 116.332, 116.314, 116.318, 116.32, 116.312,
          116.309, 116.315, 116.32, 116.291, 116.301, 116.341]

#data for HV calibration
data_HV = {'ADCmin': [0x02EB], 'DMMmin': [1.378], 'ADCmax': [0xFEB5], 'DMMmax': [120]}
df_HV = pd.DataFrame(data=data_HV)

In [3]:
def GetADCvalues(file_DACset):

    tmp_df = pd.read_csv(file_DACset)

    #remove some columns not exploitable
    tmp_df.drop(['Bus','No', 'ID (hex)', 'Message', 'ASCII'], inplace=True, axis=1)

    #save dataframe in a file to be read by the script
    filename = file_DACset.split('/')[2]
    tmp_filename = 'calibfile_fromscript_'+ filename+'.txt'
    tmp_df.to_csv(tmp_filename, sep=',', index=False, header=False, quotechar = ' ')

    #open file to read
    infile = open(tmp_filename, 'r')

    #call function which read the file and return the content ordered in a list of dictionaries
    d_list = readfile(infile)

    DACset = getDACsetperchannel(d_list)
    DACset

    my_allch_list = getADCperchannel(d_list)
    mych_average_biasV = Getch_average(my_allch_list)
    hexprint = list(map(trunchex, mych_average_biasV))


    return DACset, hexprint

In [4]:
DACset000, ADCmin = GetADCvalues(file_DACset000)
#print(DACset000)
#print(ADCmin)

DACsetFFF, ADCmax = GetADCvalues(file_DACsetFFF)

In [5]:
#build the dataframe to use to extract the calibration parameters
if (DACset000 == '0x0'):
    col0 = 'ADCmin'
else: 
    print('Error! not the expected values for the DACset')
    abort()

if (DACsetFFF == '0xfff'):
    col3 = 'ADCmax'
else: 
    print('Error! not the expected values for the DACset')
    abort()

    df_out = pd.DataFrame(list(zip(ADCmin, DMMmin, ADCmax, DMMmax)), columns = [col0, 'DMMmin' ,  col3, 'DMMmax'])
    df_out

,ADCmin,DMMmin,ADCmax,DMMmax
0,e6b6,108.752,f6c9,116.327
1,e6bb,108.755,f6cf,116.332
2,e6b4,108.740,f6c7,116.314
3,e6ad,108.744,f6bf,116.318
4,e6c2,108.744,f6d5,116.320
5,e6ba,108.744,f6c9,116.312
6,e6ae,108.735,f6c1,116.309
7,e6b7,108.741,f6c9,116.315
8,e6fa,108.745,f701,116.320
9,e6cf,108.715,f6e6,116.291


In [6]:
#convert ADCmin and ADCmax in decimal numbers
df_out = hex_to_dec(df_out)
df_out

,ADCmin,DMMmin,ADCmax,DMMmax
0,59062,108.752,63177,116.327
1,59067,108.755,63183,116.332
2,59060,108.740,63175,116.314
3,59053,108.744,63167,116.318
4,59074,108.744,63189,116.320
5,59066,108.744,63177,116.312
6,59054,108.735,63169,116.309
7,59063,108.741,63177,116.315
8,59130,108.745,63233,116.320
9,59087,108.715,63206,116.291


In [7]:
# compute gain and offset for the Bias Voltages
# a_coeff = 1.8535e-3
# b_coeff = 0
# Gain BiasV = (DMMmax - DMMmin) / (ADCmax - ADCmin ) * 1/a_coeff
# offset BiasV = ADCmin - (DMMmin - b_coeff )/ (Gain BiasV * a_coeff )

df_out = biasV_params(df_out)
df_out

,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV
0,59062,108.752,63177,116.327,0.993162,15.819142
1,59067,108.755,63183,116.332,0.993183,11.207734
2,59060,108.740,63175,116.314,0.993031,19.099551
3,59053,108.744,63167,116.318,0.993273,13.915236
4,59074,108.744,63189,116.320,0.993293,-8.324182
5,59066,108.744,63177,116.312,0.993210,4.637421
6,59054,108.735,63169,116.309,0.993031,22.383021
7,59063,108.741,63177,116.315,0.993273,2.285714
8,59130,108.745,63233,116.320,0.996067,-228.252805
9,59087,108.715,63206,116.291,0.992329,20.323786


In [8]:
# Gain biasV = math.ceil(Gain BiasV* 32768)
# Offset biasV = round(offset BiasV)  

df_out = code_offsetandgain(df_out,'biasV')
df_out

,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV,Gain_Usig BiasV,Offset_Sig BiasV
0,59062,108.752,63177,116.327,0.993162,15.819142,32544,16
1,59067,108.755,63183,116.332,0.993183,11.207734,32545,11
2,59060,108.740,63175,116.314,0.993031,19.099551,32540,19
3,59053,108.744,63167,116.318,0.993273,13.915236,32548,14
4,59074,108.744,63189,116.320,0.993293,-8.324182,32548,-8
5,59066,108.744,63177,116.312,0.993210,4.637421,32546,5
6,59054,108.735,63169,116.309,0.993031,22.383021,32540,22
7,59063,108.741,63177,116.315,0.993273,2.285714,32548,2
8,59130,108.745,63233,116.320,0.996067,-228.252805,32639,-228
9,59087,108.715,63206,116.291,0.992329,20.323786,32517,20


In [9]:
# convert following prescriptions
 # GainUsig BiasV   = hex(Gain_Usig BiasV) 
 # Offset_Sig BiasV = tohextwocompl(Offset_Sig BiasV ,15) 

df_out = convert_to_exadec(df_out, 'biasV')
df_out

,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV,Gain_Usig BiasV,Offset_Sig BiasV,Gain_hex BiasV,Offset_hex BiasV
0,59062,108.752,63177,116.327,0.993162,15.819142,32544,16,0x7f20,0x10
1,59067,108.755,63183,116.332,0.993183,11.207734,32545,11,0x7f21,0xb
2,59060,108.740,63175,116.314,0.993031,19.099551,32540,19,0x7f1c,0x13
3,59053,108.744,63167,116.318,0.993273,13.915236,32548,14,0x7f24,0xe
4,59074,108.744,63189,116.320,0.993293,-8.324182,32548,-8,0x7f24,0xfff8
5,59066,108.744,63177,116.312,0.993210,4.637421,32546,5,0x7f22,0x5
6,59054,108.735,63169,116.309,0.993031,22.383021,32540,22,0x7f1c,0x16
7,59063,108.741,63177,116.315,0.993273,2.285714,32548,2,0x7f24,0x2
8,59130,108.745,63233,116.320,0.996067,-228.252805,32639,-228,0x7f7f,0xff1c
9,59087,108.715,63206,116.291,0.992329,20.323786,32517,20,0x7f05,0x14


In [10]:
#do the same for the HV
#Compute gain and offset for the HV

# a_coeff = 1.8409e-3
# b_coeff = 0
# Gain BiasV = (DMMmax - DMMmin) / (ADCmax - ADCmin ) * 1/a_coeff
# offset BiasV = ADCmin - (DMMmin - b_coeff )/ (Gain BiasV * a_coeff )


df_HV = HV_params(df_HV)
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV
0,747,1.378,65205,120,0.999674,1.79132


In [11]:
# Gain HV = math.ceil(Gain HV* 32768)
# Offset HV = round(offset HV)  

df_HV = code_offsetandgain(df_HV,'HV')
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV,Gain_Usig HV,Offset_Sig HV
0,747,1.378,65205,120,0.999674,1.79132,32757,2


In [12]:
# convert following prescriptions
# GainUsig BiasV   = hex(Gain_Usig HV) 
# Offset_Sig BiasV = tohextwocompl(Offset_Sig HV ,15) 

df_HV = convert_to_exadec(df_HV, 'HV')
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV,Gain_Usig HV,Offset_Sig HV,Gain_hex HV,Offset_hex HV
0,747,1.378,65205,120,0.999674,1.79132,32757,2,0x7ff5,0x2


In [13]:
df_calib_biasV = df_out[['Gain_hex BiasV','Offset_hex BiasV'] ].copy()
df_calib_biasV.to_csv('calibDB/calib_biasV_SN04_v6.csv')
print(df_calib_biasV)

print('\n\n\n')
df_calib_HV = df_HV[['Gain_hex HV','Offset_hex HV'] ].copy()
df_calib_HV.to_csv('calibDB/calib_HV_SN04_v6.csv')
print(df_calib_HV)

   Gain_hex BiasV Offset_hex BiasV
0          0x7f20             0x10
1          0x7f21              0xb
2          0x7f1c             0x13
3          0x7f24              0xe
4          0x7f24           0xfff8
5          0x7f22              0x5
6          0x7f1c             0x16
7          0x7f24              0x2
8          0x7f7f           0xff1c
9          0x7f05             0x14
10         0x7f10           0xfffb
11         0x7f08             0x13




  Gain_hex HV Offset_hex HV
0      0x7ff5           0x2


In [16]:
df_calib_biasV_tocheck = df_out[['Gain_Usig BiasV', 'Offset_Sig BiasV', 'Gain_hex BiasV','Offset_hex BiasV']].copy()
df_calib_biasV_tocheck.to_csv('calibDB/calib_biasV_SN04_v6_forcheck.csv')